# 1. Loader

In [20]:
from helper import *
from algorithms import *
import menpo3d
from importlib import reload
import datetime
import numpy as np
import importlib
from scipy import sparse
import time
import matplotlib.pyplot as plt
import warnings

# %matplotlib qt

First load the BFM mean model as the target mesh.

On shark, please comment on `%matplotlib qt` and all `mesh.view()` codes, as shark does not provides these viewings. If you are on your laptop, please un-comment these and help yourself viewing 3D faces.

In [16]:
bfm_mean = loader.get_mean_model("../BFM.mat")
#bfm_mean.view()

You should see

<img src="./BFM_mean.png">

Then load the source mesh to do the transformation.

In [5]:
source = loader.get_mesh("../lsfm-ori/1/1_0001.obj")
#source.view()

You shall see
<img src="./source_mesh.png">

# 2. Test matrix norm solver

In [10]:
importlib.reload(math_helper)

<module 'helper.math_helper' from '/home/jingtao/LSFM_Pipeline/helper/math_helper.py'>

By default, we use `umfpack` solver.

In [11]:
A = np.random.random((4000,5000))
B = np.random.random((4000,300))
solver = math_helper.Solver(A, B)

start = time.time()
X = solver.solve()
error = np.sum((A @ X - B) ** 2)
print(error)

print("\n time spent: {:.1f} seconds".format(time.time() - start))

iter: 1, error: 400378.41


/home/jingtao/anaconda3/envs/lsfm/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:718: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.82e+15)
  warnings.warn(msg, UmfpackWarning)


iter: 2, error: 0.00
2.9975796744429572e-18

 time spent: 30.3 seconds


On comparison, with naive solver.

In [71]:
A = np.random.random((4000,5000))
B = np.random.random((4000,300))
solver = math_helper.Solver(A, B, solver="naive")

start = time.time()
X = solver.solve()
error = np.sum((A @ X - B) ** 2)
print(error)

print("\n time spent: {:.1f} seconds".format(time.time() - start))

iter: 1, error: 400080.04
solving linear equation...
2018-11-12 14:31:20.359657
inversing matrix
2018-11-12 14:31:35.188524
iter: 2, error: 416857598.48
solving linear equation...
2018-11-12 14:31:37.813341
inversing matrix
2018-11-12 14:31:52.670348
iter: 3, error: 416857598.48
solving linear equation...
2018-11-12 14:31:55.261406
inversing matrix
2018-11-12 14:32:10.098824
iter: 4, error: 416857598.48
solving linear equation...
2018-11-12 14:32:12.706352
inversing matrix
2018-11-12 14:32:27.537560
iter: 5, error: 416857598.48
solving linear equation...
2018-11-12 14:32:30.128560
inversing matrix
2018-11-12 14:32:44.951544
iter: 6, error: 416857598.48
solving linear equation...
2018-11-12 14:32:47.561703
inversing matrix
2018-11-12 14:33:02.409826
iter: 7, error: 416857598.48
solving linear equation...
2018-11-12 14:33:05.005904
inversing matrix
2018-11-12 14:33:19.844723
iter: 8, error: 416857598.48
solving linear equation...
2018-11-12 14:33:22.436402
inversing matrix
2018-11-12 14:

| solver  | umfpack | naive |
|------|:------------:|----:|
| time | 30.2s | 174.6s |

# 3. Non-rigid icp

First, test non-rigid icp algorithm.

In [12]:
target = loader.get_mesh("../lsfm-ori/1/1_0002.obj")
#target.view()

In [17]:
fitter = no_landmark_nicp.NonRigidIcp(stiffness_weights=[50, 20], data_weights=[None]*2, max_iter=5)
transformed_source = fitter.non_rigid_icp(source, bfm_mean)

/home/jingtao/anaconda3/envs/lsfm/lib/python3.5/site-packages/vtk/util/numpy_support.py:134: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


Epoch 1 with stiffness 50


/home/jingtao/anaconda3/envs/lsfm/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:718: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 3.12e+16)
  warnings.warn(msg, UmfpackWarning)


 - 1 regularized_error: 125643.643  


/home/jingtao/anaconda3/envs/lsfm/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:560: UmfpackWarning: Singular matrix
  warnings.warn('Singular matrix', UmfpackWarning)
/home/jingtao/anaconda3/envs/lsfm/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:709: UmfpackWarning: Zeroing nan and inf entries...
  warnings.warn('Zeroing nan and inf entries...', UmfpackWarning)
/home/jingtao/anaconda3/envs/lsfm/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:714: RuntimeWarning: divide by zero encountered in double_scalars
  econd = 1.0 / self.info[UMFPACK_RCOND]
/home/jingtao/anaconda3/envs/lsfm/lib/python3.5/site-packages/scikits/umfpack/umfpack.py:718: UmfpackWarning: (almost) singular matrix! (estimated cond. number: inf)
  warnings.warn(msg, UmfpackWarning)


 - 2 regularized_error: 125643.643  
 - 3 regularized_error: 6.096  
 - 4 regularized_error: 68.296  
 - 5 regularized_error: 69.013  
Epoch 2 with stiffness 20
 - 1 regularized_error: 18.755  
 - 2 regularized_error: 39.527  
 - 3 regularized_error: 34.161  
 - 4 regularized_error: 13.974  
 - 5 regularized_error: 17.149  


After 2 epoches with each of 5 iterations, we could see
<img src="./face_after_5_iters.png">

Now store the mesh.

In [18]:
import pickle as pkl

with open('tr.pkl', 'wb') as fw:
    pkl.dump({'points': transformed_source.points, 'trilist': transformed_source.trilist}, fw)